In [1]:
import hpsv2
import torch
import json
import clip
from PIL import Image
import os
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from pipeline_rf import RectifiedFlowPipeline
import random
from diffusers import AutoPipelineForText2Image
import json
import generate_swift as gs
from generate import generate_single_image, load_model
from dataclasses import dataclass


Failed to get repository contents: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/datasets/zhwang/HPDv2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f1163f8c460>, 'Connection to huggingface.co timed out. (connect timeout=None)'))


/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
2024-03-03 19:54:31.539871: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN whe

In [ ]:
@dataclass
class TestConfig:
    device = "cuda:1"
    ours_steps = 8
    ours_hpsv2_path = "/data/liutao/data/ours_36k_8i_hpsv2"
    ours_coco_path = "/data/liutao/data/ours_36k_8s_coco"
    clip_model_id = "ViT-L/14@336px"
    ours_model_id = "/data/20231212/SwiftBrush_reproduce_se_parallel/checkpoints_20240228/vsd_global_step36000_8nis.pth"
    ours_base_path = "/data/"
    coco_caption_path = "/data/dataset/coco2014-val/annotations/captions_val2014.json"
    caption_num = 30000
    seed = 2024
config = TestConfig()

### load model

In [4]:
clip_model, clip_preprocess = clip.load(config.clip_model_id)
clip_model = clip_model.to(config.device)

In [5]:
vae, tokenizer, text_encoder, unet, scheduler, alphas = load_model(config.ours_base_path, config.ours_model_id, config.device)

[INFO] loading student unet checkpoint


/home/liutao/miniconda3/envs/instaflow/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


## clip score

In [7]:
def get_clip_score(image, text):
    # Load the pre-trained CLIP model and the image

    # Preprocess the image and tokenize the text
    image_input = clip_preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text], truncate=True)
    
    # Move the inputs to GPU if available
    image_input = image_input.to(config.device)
    text_input = text_input.to(config.device)
    
    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features = clip_model.encode_text(text_input)
    
    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()
    
    return clip_score

### load coco30k_caption

In [8]:
coco_f = open(config.coco_caption_path)
coco_annotations = json.load(coco_f)
captions = []
for annotation in coco_annotations['annotations']:
    caption = annotation['caption']
    captions.append(caption)
coco_f.close()
random.seed(config.seed)
captions_30k = random.choices(captions, k=config.caption_num)
print(len(captions_30k),captions[0],captions[1])

30000 A bicycle replica with a clock as the front wheel. A black Honda motorcycle parked in front of a garage.


In [9]:
count = 0
total_score = 0
for case_number, prompt in enumerate(captions_30k):
    image = generate_single_image(network=(vae, tokenizer, text_encoder, unet, scheduler),prompt=prompt,seed=2024,num_inference_steps=ours_steps)
    score = get_clip_score(image, prompt)
    save_name = str(count)+".jpg"
    image.save(os.path.join(config.ours_coco_path,save_name))
    total_score += score
    count += 1
    if count % 1000 == 0:
        print("current num:",count,"current avg clip score:",total_score/count)
print(f"AVG CLIP Score: {total_score/count}")

current num: 1000 current avg clip score: 0.2615880126953125
current num: 2000 current avg clip score: 0.2610256958007813
current num: 3000 current avg clip score: 0.260946533203125
current num: 4000 current avg clip score: 0.26053231811523436
current num: 5000 current avg clip score: 0.2607731689453125
current num: 6000 current avg clip score: 0.2605646769205729
current num: 7000 current avg clip score: 0.26062793840680804
current num: 8000 current avg clip score: 0.26105057525634767
current num: 9000 current avg clip score: 0.2608954467773438
current num: 10000 current avg clip score: 0.2608013977050781
current num: 11000 current avg clip score: 0.2607430586381392
current num: 12000 current avg clip score: 0.2606460622151693
current num: 13000 current avg clip score: 0.2606663771409255
current num: 14000 current avg clip score: 0.26059676252092634
current num: 15000 current avg clip score: 0.26063055419921877
current num: 16000 current avg clip score: 0.26057498550415037
current num:

## hpsv2 score

In [ ]:
all_prompts = hpsv2.benchmark_prompts('all') 
for style, prompts in all_prompts.items():
    for idx, prompt in enumerate(prompts):
        image = generate_single_image(network=(vae, tokenizer, text_encoder, unet, scheduler),prompt=prompt,seed=2024,num_inference_steps=ours_steps)
        image.save(os.path.join(config.ours_hpsv2_path, style, f"{idx:05d}.jpg")) 

In [ ]:
hpsv2.evaluate(config.ours_hpsv2_path) 